In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()
for table in tables:
    print(f"----{table}----")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"])
latest = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
latest=latest[0]
latest

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
dt_latest = pd.to_datetime(latest, format="%Y-%m-%d")
dt_year = dt_latest - timedelta(days=365)
dt_year = dt_year.strftime("%Y-%m-%d")

# Perform a query to retrieve the data and precipitation scores
percipitation_query = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= dt_year).\
    order_by(Measurement.date.desc()).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(percipitation_query, columns=['Date', 'Precipitation'])
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
df.set_index('Date', inplace=True)

fig, ax = plt.subplots()
ax.bar(df.index, df['Precipitation'], align='center')
ax.xaxis_date()
fig.autofmt_xdate()
plt.title('Precipitation by Date')
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_query1 = session.query(Station.station, func.count('*')).\
    group_by(Station.station).all()
len(station_query1)

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_query2 = session.query(Measurement.station, func.count('*')).\
    group_by(Measurement.station).\
    order_by(func.count('*').desc()).all()  
most_active_station = station_query2[0][0]
station_query2

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station_max = session.query(func.max(Measurement.tobs)).filter(Measurement.station == most_active_station).first()
station_min = session.query(func.min(Measurement.tobs)).filter(Measurement.station == most_active_station).first()
station_avg = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == most_active_station).first()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
station_date = session.query(Measurement.date).filter(Measurement.station == most_active_station).order_by(Measurement.date.desc()).first()
station_date = pd.to_datetime(station_date, format="%Y-%m-%d")
station_year = station_date - timedelta(days=365)
station_year = station_year.strftime("%Y-%m-%d")

temperature_query = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.date >= dt_year).\
    filter(Measurement.station == most_active_station).\
    order_by(Measurement.date.desc()).all()

df2 = pd.DataFrame(temperature_query, columns=['Date', 'Temperature'])
df2['Date'] = pd.to_datetime(df2['Date'], format="%Y-%m-%d")
df2.set_index('Date', inplace=True)
df2

plt.hist(df2['Temperature'], bins=12)
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.show()

# Close session

In [ ]:
# Close Session
session.close()